In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132734")
exp = Experiment(workspace=ws, name="udacity-project-hyper")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132734
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-132734


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-2"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing Comput Target")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [48]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
import os

# Specify parameter sampler
ps1 = RandomParameterSampling(
     {"--C": loguniform(-3,2), "--max_iter": choice(1,5,10,20,30,40,50,80,100,200,400,800,1000)}
)

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
arguments = {"--C": 1.0, "--max_iter": 100}
current_dir = os.getcwd()
est = SKLearn(source_directory=current_dir, 
              entry_script="train.py",
              compute_target = compute_target,
              script_params = arguments)

### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps1,
    primary_metric_name="Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=2
)

In [49]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run_hyper = exp.submit(config=hyperdrive_config)
### YOUR CODE HERE ###

In [ ]:
run_hyper.wait_for_completion(show_output=True)

RunId: HD_f5269347-240f-45d5-8554-dc1c5b9aeb9e
Web View: https://ml.azure.com/experiments/udacity-project-hyper/runs/HD_f5269347-240f-45d5-8554-dc1c5b9aeb9e?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-132652/workspaces/quick-starts-ws-132652

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-31T17:23:54.124685][API][INFO]Experiment created<END>\n""<START>[2020-12-31T17:23:54.748461][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2020-12-31T17:23:54.890359][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-31T17:23:55.6981247Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
best_run = run_hyper.get_best_run_by_primary_metric()

In [ ]:
print(best_run.get_file_names())

In [ ]:
model = best_run.register_model(workspace=ws, model_name="best-hyper-model", model_path="outputs/model.joblib")

In [26]:
dir(best_run)

['DELIM',
 'EXPERIMENT_PATH',
 'RUN_PATH',
 'WORKSPACE_FMT',
 '_DEFAULT_GET_CONTENT_TIMEOUT',
 '_RUNSOURCE_PROPERTY',
 '_WAIT_COMPLETION_POLLING_INTERVAL_MAX',
 '_WAIT_COMPLETION_POLLING_INTERVAL_MIN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cleanup',
 '_client',
 '_container',
 '_context_manager',
 '_create',
 '_download_artifact_contents_to_string',
 '_dto_to_run',
 '_experiment',
 '_experiment_url',
 '_get_base_info_dict',
 '_get_blob_azureml_datastore',
 '_get_last_log_primary_instance',
 '_get_logs',
 '_get_outputs_datapath',
 '_get_portal_domain',
 '_heartbeat',
 '_identity',
 '_kill',
 '_load_scope',
 '_log_context',
 '_logge

In [22]:
import joblib
# Get your best run and save the model from that run.
best_run = run_hyper.get_best_run_by_metric()
model = best_run.register_model(model_name="best-hyper-model", "outputs/model")

SyntaxError: positional argument follows keyword argument (<ipython-input-22-d5c24c770e18>, line 4)

In [4]:
exp_auto = Experiment(workspace=ws, name="udacity-project-automl")
run_auto = exp_auto.start_logging()

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", header=True, validate=True)

In [7]:
# Somehow the loading of the local module "train.py" doesn't work instead the clean_data method is directly initilized in the 
# udacity-project Jupyter Notebook
import pandas as pd

def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12}
    weekdays = {"mon":1, "tue":2, "wed":3, "thu":4, "fri":5, "sat":6, "sun":7}

    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
    x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
    contact = pd.get_dummies(x_df.contact, prefix="contact")
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix="education")
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

    y_df = x_df.pop("y").apply(lambda s: 1 if s == "yes" else 0)
    return x_df, y_df

In [8]:
# from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y

In [12]:
from collections import Counter 
Counter(x['y'])

Counter({0: 29258, 1: 3692})

In [13]:
# Somehow the registered Dataset cannot be used for AutoML 
from azureml.core import Dataset
dataset_training = Dataset.from_pandas_dataframe(x)
dataset_training = dataset_training.register(workspace=ws, name="auto-ml-training-data", description="example exercise data")

In [25]:
# To be able to use a AutoML model on a Dataset it is necessary to save it e.g. in Azureml Blob Storage and then load it from 
# there
import os
from azureml.core import Dataset, Datastore, Workspace
os.makedirs("data", exist_ok=True)

dataset_path = "data/train_set.csv"
x.to_csv(dataset_path)
datastore = ws.get_default_datastore()
x.to_csv(dataset_path)
datastore.upload(src_dir="data", target_path="data")

Uploading an estimated of 1 files
Uploading data/train_set.csv
Uploaded data/train_set.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_62b99680338c477ea95cd78cf6b7a54e

In [26]:
train_data = Dataset.Tabular.from_delimited_files(path = [(datastore, ("data/train_set.csv"))])

In [28]:
train_data.to_pandas_dataframe()

,Column1,age,marital,default,housing,loan,month,day_of_week,duration,campaign,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,0,57,1,0,0,1,5,1,371,1,...,0,0,0,0,1,0,0,0,0,0
1,1,55,1,0,1,0,5,4,285,2,...,1,0,0,0,0,0,0,0,1,0
2,2,33,1,0,0,0,5,5,52,1,...,0,0,0,1,0,0,0,0,0,0
3,3,36,1,0,0,0,6,5,355,4,...,1,0,0,0,1,0,0,0,0,0
4,4,27,1,0,1,0,7,5,189,2,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,32945,56,1,0,0,1,7,1,116,1,...,0,1,0,0,0,0,0,0,0,0
32946,32946,37,1,0,0,1,7,5,69,7,...,0,0,0,0,0,0,0,1,0,0
32947,32947,26,0,0,0,0,5,2,135,4,...,0,0,0,0,0,0,0,1,0,0
32948,32948,31,0,0,0,0,4,1,386,1,...,0,0,0,1,0,0,0,0,0,0


In [29]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run
run_auto = exp_auto.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster-2 with default configuration
Running on remote compute: cpu-cluster-2
Parent Run ID: AutoML_c0ef617f-f985-4f63-b8ed-7e33e5df902e

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------

In [ ]:
run_auto.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###